In [1]:
import os

# ***IMPORTANT***
# Be sure to modify the 'qiime2-20XX.X' to indicate the version of qiime you are using
os.environ['PATH'] = '/home/jovyan/my-conda-envs/qiime2-2019.1/bin:' + os.environ['PATH']
os.environ['PROJ_LIB'] = '/home/jovyan/my-conda-envs/qiime2-2019.1/share/proj'

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from IPython.core.display import display, HTML
from ipyparallel import Client
import math
import numpy as np

# SQL Query imports
from sqlalchemy import create_engine, engine

import biota
from biota.database import get_data
from biota.util import assign_formations, rarefy
import biota.plot as bpl
from sourcetracker import gibbs

%matplotlib inline

In [3]:

# Step 1: Connect to Database
host = 'biota-pg.cldeduw9cmn5.us-east-1.rds.amazonaws.com'
drivername='postgresql'
username='wilkins'
password='glassy_tinware_madonna'
port=5432
database='biotadb'

con = create_engine(engine.url.URL(drivername=drivername,
                                   username=username,
                                   password=password,
                                   port=port,
                                   database=database,
                                   host=host))

In [29]:
# Step 2: Define the tables and columns we want returned
# field columns we are interested in
field_columns = ['collectiondate', 'fieldsampletype']
field_columns = ', '.join(['field.{}'.format(e) for e in field_columns])

# well columns we are interested in
well_columns = ['wellname','company', 'well_name_company', 'targetformation']
well_columns = ', '.join(['well.{}'.format(e) for e in well_columns])

# sequence columns we are interested in
sequence_columns = ['necplate']
sequence_columns = ', '.join(['sequence.{}'.format(e) for e in sequence_columns])

In [30]:
## These wells are in the order for gunbarrel view
target_wells = ["silvertip.76.7.unit.a.3h",
                    "silvertip.76.7.unit.a.1h",
                    "silvertip.76.7.unit.a.2h",
                    "silvertip.76.7.unit.a.4h",
                    "silvertip.76.7.unit.a.5h",
                    "silvertip.76.7.unit.a.6h",
                    "silvertip.76.7.unit.a.7h",
                    "silvertip.76.7.unit.a.8h",
                    "silvertip.76.7.unit.a.9h",
                    "silvertip.76.7.unit.a.10h",
                    "silvertip.76.7.unit.b.2h",
                    "silvertip.76.7.unit.b.3h"]

# Step 3: Format the query
query = """
SELECT {}, {}, {}
FROM field
LEFT JOIN bridge
    ON field.fieldsampleuuid = bridge.fieldsampleuuid 
LEFT JOIN sequence
    ON bridge.sequencesampleuuid = sequence.sequencesampleuuid
INNER JOIN well
    ON field.welluuid=well.welluuid
    
WHERE company = 'anadarko.petroleum.corporation'
AND wellname IN ({})
AND usage = 'customer'
AND fieldsampletype = 'produced.fluid'
""".format(field_columns, sequence_columns, well_columns,
           (', '.join("'" + item + "'" for item in target_wells)))

In [32]:
customer_samples = pd.read_sql_query(query, con)

In [43]:
customer_samples.groupby(['wellname']).size()

wellname
silvertip.76.7.unit.a.10h    103
silvertip.76.7.unit.a.1h      85
silvertip.76.7.unit.a.2h      75
silvertip.76.7.unit.a.3h      84
silvertip.76.7.unit.a.4h      90
silvertip.76.7.unit.a.5h     104
silvertip.76.7.unit.a.6h      90
silvertip.76.7.unit.a.7h     101
silvertip.76.7.unit.a.8h     111
silvertip.76.7.unit.a.9h     101
silvertip.76.7.unit.b.2h      87
silvertip.76.7.unit.b.3h      92
dtype: int64

In [48]:
customer_samples['wellname'] = 'silvertip.76.7.unit.a.6h']['collectiondate']

SyntaxError: invalid syntax (<ipython-input-48-9613e353215e>, line 1)

In [72]:
df = customer_samples.loc[customer_samples['wellname'] == 'silvertip.76.7.unit.a.1h']

In [73]:
fofo = df.collectiondate.unique()

In [74]:
print (fofo)

[datetime.date(2018, 11, 1) datetime.date(2018, 11, 8)
 datetime.date(2018, 12, 5) datetime.date(2018, 12, 19)
 datetime.date(2019, 2, 14) datetime.date(2019, 3, 5)
 datetime.date(2019, 3, 19) datetime.date(2019, 5, 6)
 datetime.date(2019, 5, 15) datetime.date(2019, 5, 20)
 datetime.date(2019, 5, 29) datetime.date(2019, 6, 11)
 datetime.date(2019, 6, 15) datetime.date(2019, 6, 19)
 datetime.date(2019, 6, 24) datetime.date(2019, 7, 11)
 datetime.date(2019, 7, 18)]


In [55]:
df.groupby['collectiondate']

TypeError: 'method' object is not subscriptable

In [63]:
datesdf = {}
for well in target_wells:
    df = customer_samples.loc[customer_samples['wellname'] == well]
    datesdf[well] =  df.collectiondate.unique()

In [97]:
dfdate2 = datesdf['silvertip.76.7.unit.b.3h'].strftime('%m/%d/%Y')

AttributeError: 'numpy.ndarray' object has no attribute 'strftime'

In [95]:
dfdate = pd.to_datetime(datesdf['silvertip.76.7.unit.b.3h'])

In [96]:
dfdate

DatetimeIndex(['2019-01-09', '2019-01-17', '2019-03-12', '2019-03-19',
               '2019-03-21', '2019-04-04', '2019-04-30', '2019-04-23',
               '2019-05-06', '2019-05-15', '2019-05-20', '2019-05-29',
               '2019-06-11', '2019-06-15', '2019-06-19', '2019-06-24',
               '2019-07-11', '2019-07-17'],
              dtype='datetime64[ns]', freq=None)

In [44]:
#grouped = customer_samples.groupby('wellname','collectiondate')

#customer_samples = grouped.aggregate(lambda x: tuple(x))

#customer_samples['grouped'] = customer_samples['wellname'] + customer_samples['collectiondate']

ValueError: No axis named collectiondate for object type <class 'pandas.core.frame.DataFrame'>

In [42]:
customer_samples.groupby(['wellname', 'collectiondate']).size()

wellname                   collectiondate
silvertip.76.7.unit.a.10h  2018-12-12         5
                           2019-01-09         4
                           2019-01-17         5
                           2019-01-30         5
                           2019-03-12         5
                           2019-03-19         5
                           2019-03-21         4
                           2019-04-04         5
                           2019-04-17         5
                           2019-04-23         5
                           2019-04-30         5
                           2019-05-06         5
                           2019-05-15         5
                           2019-05-20         5
                           2019-05-29         5
                           2019-06-11         5
                           2019-06-15         5
                           2019-06-19         5
                           2019-06-24         5
                           2019-07-11         

In [40]:
print(listd)

In [6]:

# Step 4: Submit the query to the database to get the customer samples
customer_samples = pd.read_sql_query(query, con)

# Step 5: Tidy up and GROUP data
# tidy up
customer_samples.rename(columns={'collectionlocation': 'fieldsamplecollectionlocation',
                        'wellname': 'wellname'}, inplace=True)

# Import default groupings
from biota.database import DEFAULT_SQL_GROUPINGS
DEFAULT_SQL_GROUPINGS = {key.lower(): value for key, value in DEFAULT_SQL_GROUPINGS.items() \
                         if key in customer_samples.columns}
DEFAULT_SQL_GROUPINGS['labid'] = 'first'
DEFAULT_SQL_GROUPINGS['necfieldsampleuuid'] = 'first'
DEFAULT_SQL_GROUPINGS['forseqrun'] = 'first'
DEFAULT_SQL_GROUPINGS['firstproductiondate'] = 'first'
DEFAULT_SQL_GROUPINGS['targetformation'] = 'first'

## Grab all the MDs and TVDs contributing to a single sequencing sample and save them as a list
mds = customer_samples.groupby('sequencesampleuuid')['md'].apply(list).rename('md_list')
tvds = customer_samples.groupby('sequencesampleuuid')['tvd'].apply(list).rename('tvd_list')

## Warn if there are field samples that haven't been sequenced 
print("There are {} field samples that haven't been sequenced".format(customer_samples.loc[customer_samples.sequencesampleuuid.isnull()
                                                                                    ].fieldsampleuuid.nunique()))
## group by sequencesampleuuid according to DEFAULT_GROUPINGS
customer_samples = customer_samples.groupby('sequencesampleuuid').agg(DEFAULT_SQL_GROUPINGS)

## Warn if there are sequence samples that weren't sent for sequencing
print("There are {} sequence samples that weren't sent for sequencing".format(customer_samples.loc[customer_samples.forseqrun != 1].shape[0]))

## concat
customer_samples = pd.concat([customer_samples, mds, tvds], axis=1)
customer_samples.index = customer_samples.index.astype(str)

There are 0 field samples that haven't been sequenced
There are 43 sequence samples that weren't sent for sequencing


In [ ]:
# Use modifed query code from above
well_query = {'Company': 'occidental.petroleum.corporation', 
              'Well': 'peck.state.258.6h'}
samples, data, features = get_sample_data(well_query)
samples.CollectionDate = pd.to_datetime(samples.CollectionDate).apply(lambda x: x.strftime('%Y-%m-%d'))

In [8]:
# Format a parenthetical list of each item in the nec plates
nec_plates = customer_samples.necplate.dropna().unique().tolist()

## Some older samples may be associated with NEC samples that are not located on an NEC plate.
## NEC samples that on the same PCR plate as a customer's samples will "look" like the 
## customer samples due to well-to-well contamination. Be cautious when using such samples.
## You may need to use a different NEC plate for the QC process
nec_plates = [n for n in nec_plates if n.find('nec')!=-1 or n.find('NEC')!=-1]

## As we move towards NEC field sample uuids, NEC plates will be phased out
if nec_plates == []:
    print("There aren't any associated NEC plates")
    nec_plates = ['None']

# Format a parenthetical list of each item in the nec field sample uuids
## We need to drop empty cells, cast non-empty cells to strings, and obtain all unique field sample uuids
necfielduuids = customer_samples.necfieldsampleuuid.dropna().astype(str).unique().tolist()
necfielduuids = [x for x in necfielduuids]

## Older samples may not have an assoicated NEC field sample uuid
if necfielduuids == []:
    print("There aren't any associated NEC field uuids")
    necfielduuids = ['11111111-1111-1111-1111-111111111111']

in_nec_plate = (', '.join("'" + item + "'" for item in nec_plates))
in_nec_fuuid = (','.join("'" + item + "'" for item in necfielduuids))

# create query
nec_query = """
SELECT {}, {}
FROM sequence
INNER JOIN bridge
    ON sequence.sequencesampleuuid = bridge.sequencesampleuuid 
INNER JOIN field
    ON bridge.fieldsampleuuid = field.fieldsampleuuid
WHERE
    (
        extractionplate IN ({})
    OR
        field.fieldsampleuuid IN ({})
    )
AND 
    sequencesampletype = 'negative.extraction.control'
AND
    usage = 'control'
""".format(field_columns, sequence_columns, in_nec_plate, in_nec_fuuid)

# get back nec dataframe
necs = pd.read_sql_query(nec_query, con)

from biota.database import DEFAULT_SQL_GROUPINGS
DEFAULT_SQL_GROUPINGS = {key.lower(): value for key, value in DEFAULT_SQL_GROUPINGS.items()
                         if key in necs.columns}
DEFAULT_SQL_GROUPINGS['fieldsampleuuid'] = 'first'

necs = necs.groupby('sequencesampleuuid').agg(DEFAULT_SQL_GROUPINGS)

necs.index = necs.index.astype(str)

In [9]:
## Example checks:
print("{} NEC plates are missing".format(len(set(customer_samples.necplate.dropna().unique()
                                                ).difference(set(necs.extractionplate.unique())))))
print("{} NEC field sample uuids are missing".format(len(set(customer_samples.necfieldsampleuuid.dropna().unique()
                                                            ).difference(set(necs.fieldsampleuuid.unique())))))

0 NEC plates are missing
0 NEC field sample uuids are missing


In [99]:
samples = pd.concat([customer_samples, necs], sort=False)
samples.fieldsampleuuid = samples.fieldsampleuuid.astype(str)

In [107]:
samples.loc[mp2.loc[mp2.Unknown>=0.8].index, 'bgpass'] = 1

NameError: name 'mp2' is not defined

In [103]:
# necs = samples.loc[samples.extractionplate.isin(nec_plates)].copy()
wells = samples.loc[(samples.fieldsampletype=='produced.fluid')].copy()

In [104]:
wells.head()

,collectiondate,fieldsampletype,necplate,wellname,company,well_name_company,targetformation,md,tvd,pcryield,usage,sequencesampletype,barcodesequence,extractionplate,sequencinggroup,sequencingrunid,fieldsampleuuid
0,2019-01-09,produced.fluid,nec.plate.405.20181226,silvertip.76.7.unit.a.10h,anadarko.petroleum.corporation,Silvertip 76 7 Unit A 10H,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan
1,2018-08-30,produced.fluid,nec.plate.300.20180907,silvertip.76.7.unit.a.2h,anadarko.petroleum.corporation,Silvertip 76 7 Unit A 2H,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan
2,2018-08-30,produced.fluid,nec.plate.300.20180907,silvertip.76.7.unit.a.2h,anadarko.petroleum.corporation,Silvertip 76 7 Unit A 2H,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan
3,2018-08-30,produced.fluid,nec.plate.300.20180907,silvertip.76.7.unit.a.2h,anadarko.petroleum.corporation,Silvertip 76 7 Unit A 2H,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan
4,2018-08-30,produced.fluid,nec.plate.300.20180907,silvertip.76.7.unit.a.2h,anadarko.petroleum.corporation,Silvertip 76 7 Unit A 2H,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan


In [11]:
samples['ip'] = [days.days for days in (samples['collectiondate']\
                                                 - samples['firstproductiondate'])]

In [12]:

# quick check of the samples
samples.groupby(['sequencesampletype']).size()

sequencesampletype
negative.extraction.control     948
oil.water                      1123
dtype: int64

In [13]:
# Step 8: Get the data and features
data, features = get_data({'_id': {'$in': samples.index.tolist()}})

The filtering function dropped 0 samples and 24443 features


In [14]:
## Get fluids by well and date
samples['welldate'] = samples.wellname + "_" + samples.collectiondate.astype(str)

fluids = samples.loc[(samples.fieldsampletype=='produced.fluid')].copy()

## Sometimes, sequences per sample will change because more features are passing the filter function,
## so recalculate
fluids['sequencespersample'] = data.reindex(fluids.index).sum(axis=1)

In [98]:
#samples.groupby("wellname")['collectiondate'].head()

In [25]:
samples.head()

,basin,collectiondate,company,fieldsamplecollectionlocation,fieldsampletype,md,play,state,tvd,wellname,...,labid,necfieldsampleuuid,forseqrun,firstproductiondate,targetformation,md_list,tvd_list,fieldsampleuuid,ip,welldate
sequencesampleuuid,,,,,,,,,,,,,,,,,,,,,
0015d21a-e423-4b76-8209-fd2f2d1e7ef1,permian.basin,2019-05-20,anadarko.petroleum.corporation,wellhead,produced.fluid,NaN,delaware,texas,NaN,silvertip.76.7.unit.a.7h,...,bio.01.9962,4c31d540-651b-4f11-a17a-e0eb8609a53e,1.0,2018-11-14,None,[None],[None],nan,187.0,silvertip.76.7.unit.a.7h_2019-05-20
007a3161-4bed-415c-90e0-d6df6d3f41bf,permian.basin,2018-12-19,anadarko.petroleum.corporation,wellhead,produced.fluid,NaN,delware,texas,NaN,silvertip.76.7.unit.a.3h,...,bio.02.1553,0e16bfe0-b6f1-4522-b77a-9311dd234492,1.0,2018-09-25,None,[None],[None],nan,85.0,silvertip.76.7.unit.a.3h_2018-12-19
009c98a1-2f45-4446-8d6e-ce11b098a16d,permian.basin,2018-08-30,anadarko.petroleum.corporation,separator,produced.fluid,NaN,delware,texas,NaN,silvertip.76.7.unit.a.2h,...,ana.05.0141,None,1.0,2018-08-29,None,[None],[None],nan,1.0,silvertip.76.7.unit.a.2h_2018-08-30
010667cc-aae4-461e-a263-3a125a39958a,permian.basin,2019-04-30,anadarko.petroleum.corporation,wellhead,produced.fluid,NaN,delware,texas,NaN,silvertip.76.7.unit.b.3h,...,bio.02.1122,7238f7a5-2fe4-4334-9d36-8816928c83d8,1.0,2018-12-19,None,[None],[None],nan,132.0,silvertip.76.7.unit.b.3h_2019-04-30
011cc09c-adf6-4ac8-90eb-e8b1b3e8e338,permian.basin,2019-06-19,anadarko.petroleum.corporation,wellhead,produced.fluid,NaN,delaware,texas,NaN,silvertip.76.7.unit.a.7h,...,bio.04.0444,8ba74dae-b971-4759-87fd-ce38910812a3,1.0,2018-11-14,None,[None],[None],nan,217.0,silvertip.76.7.unit.a.7h_2019-06-19


In [27]:
samples.groupby(['fieldsampletype', 'wellname', 'collectiondate']).size()

fieldsampletype  wellname                   collectiondate
produced.fluid   silvertip.76.7.unit.a.10h  2018-12-12         5
                                            2019-01-09         4
                                            2019-01-17         5
                                            2019-01-30         5
                                            2019-03-12         5
                                            2019-03-19         5
                                            2019-03-21         4
                                            2019-04-04         5
                                            2019-04-17         5
                                            2019-04-23         5
                                            2019-04-30         5
                                            2019-05-06         5
                                            2019-05-15         5
                                            2019-05-20         5
                               